<a href="https://colab.research.google.com/github/noambassat/SupremeCourtClassifier/blob/main/5_second_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embedding + first classifier

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, cross_val_score, StratifiedKFold,train_test_split,cross_val_predict
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score,make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler


In [5]:
data = pd.read_csv("full_final_df.csv")

In [17]:
data["outcome_category"].unique()

array([1, 0, 2])


## Training

In [7]:

tokenizer = AutoTokenizer.from_pretrained("dean-ai/sentence_transformer_Legal-heBERT")
model = AutoModel.from_pretrained("dean-ai/sentence_transformer_Legal-heBERT")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
# פונקציה שמקבלת טקסט ומחזירה את האימבדינג כוקטור
def get_embeddings(text):
    # המרת הטקסט לפורמט מתאים למודל
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    # חישוב האימבדינג של המודל
    outputs = model(**inputs)
    # חישוב ממוצע הוקטורים עבור כל טקסט
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings[0]

In [9]:
# פונקציה לחיתוך הטקסט בהתאם לכללים
def trim_text(text):
    min_length = 500  # אורך מינימלי לשמירה
    last_sen = 30     # מספר תווים להסרה מסוף הטקסט

    # שמירת אורך מקורי לאבחון
    original_length = len(text)

    # הסרת מספר תווים מסוף הטקסט
    text = text[:-last_sen]

    # חיתוך ל-min_length האחרונים אם הטקסט ארוך יותר מהמינימום
    if len(text) > min_length:
        text = text[-min_length:]

    # הדפסת פידבק רק אם הטקסט עבר שינוי
    if len(text) != original_length:
        print(f"Trimmed Text (Original Length: {original_length}, Trimmed Length: {len(text)}):")
        formatted_text = "\n".join([text[i:i+80] for i in range(0, len(text), 80)])
        print(formatted_text)
        print("##*******************************************************####")
    return text


# הסרת שורות עם ערכים חסרים בעמודת הטקסט או בעמודת המטרה
data = data.dropna(subset=["גוף המסמך", "outcome_category"])

# חיתוך הטקסטים בעמודת "גוף המסמך"
data["גוף המסמך"] = data["גוף המסמך"].apply(trim_text)

# # שמירת התוצאה לקובץ
# file_path = 'final_trimmed_data.csv'
# data.to_csv(file_path, index=False, encoding='utf-8')

# print(f"הקובץ נשמר בהצלחה בנתיב: {file_path}")


Streaming output truncated to the last 5000 lines.
Trimmed Text (Original Length: 5028, Trimmed Length: 500):
סים השונים ואנו תקווה כי לאחר שירצה המערער את עונשו וייתןאת הדין על מעשיו תטוהר 
גם האווירה הקשה שנוצרה במשפחה ויווצר פתח להידברותוהתקרבות מחודשת. המשיבה תוודא כ
י המערער יופנה למב"ןעם קליטתו בשירות בתי הסוהר. הערעור נדחה. ניתן היום, ט\' בתמו
ז, תשס"ו(5.7.06). ש ו פ ט ת ש ו פט ש ו פ ט המשיב יתייצב לריצוי עונשו במזכירות בי
תהמשפט המחוזי בתל-אביב ביום 1.8.06 בשעה 10:00. ש ו פ ט ת ש ו פט ש ו פ ט ________
_________________העותק כפוף לשינויי עריכה וניסוח. 06051880_B04.doc עכמרכז מידע, 
טל\' 02-6593666 ; את
##*******************************************************####
Trimmed Text (Original Length: 890, Trimmed Length: 500):
ל המערער בע"פ 7852/06 הועמד על 180,000 ₪ או 5 חודשי מאסר תמורתו. יתרת הקנס שטרם 
שולמה, תשולם בהתאם ובהמשך למתווה התשלומים שקבע בית המשפט המחוזי בגזר דינו, דהיינ
ו ב-22 תשלומים חודשיים שווים ורצופים, החל מיום 1.10.07 וביום הראשון לכל אחד מ-21
 החודשים שבעקבותיו. 

In [10]:
from sklearn.metrics import precision_recall_curve, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, make_scorer
from sklearn.model_selection import cross_val_predict, cross_validate
import joblib


# עדכון עמודת המטרה ואימבדינג לאחר חיתוך הטקסטים
y = data["outcome_category"].values
embeddings = np.array([get_embeddings(text) for text in data["גוף המסמך"]])

In [18]:
from sklearn.model_selection import cross_val_predict, cross_validate  # במקום מ-sklearn.metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
import joblib


# ווידוא שעמודת outcome_category קיימת
if 'outcome_category' not in data.columns:
    raise ValueError("The 'outcome_category' column is missing in the dataset!")

# ווידוא שכל הערכים ב-outcome_category הם מתוך הקטגוריות 0, 1, ו-3
valid_categories = {0, 1, 2}  # כולל את קטגוריה 2
invalid_categories = set(data['outcome_category'].unique()) - valid_categories
if invalid_categories:
    raise ValueError(f"The following categories are invalid: {invalid_categories}")

# הגדרת y_numeric לפי עמודת התוויות
y_numeric = data['outcome_category'].values

# ווידוא שהתוויות הן רק 0, 1, או 3
print("Unique categories in 'y_numeric':", np.unique(y_numeric))

# חישוב התפלגות הקטגוריות המקורית
unique, counts = np.unique(y_numeric, return_counts=True)
class_distribution = dict(zip(unique, counts))
print("Original class distribution:", class_distribution)

# הגדרת יחס הדגימה הנדרש לכל קטגוריה
sampling_strategy = {
    0: min(class_distribution[0], int(class_distribution[0] * 0.6)),  # קטגוריה 0
    1: min(class_distribution[1], int(class_distribution[0] * 0.2)),  # קטגוריה 1
    2: min(class_distribution[2], int(class_distribution[0] * 0.2))   # קטגוריה 2
}
print("Sampling strategy:", sampling_strategy)

# ביצוע undersampling
undersampler = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(embeddings, y_numeric)

# שמירת האינדקסים המקוריים
resampled_indices = undersampler.sample_indices_
joblib.dump(resampled_indices, 'resampled_indices.pkl')

# חישוב התפלגות לאחר undersampling
unique_resampled, counts_resampled = np.unique(y_resampled, return_counts=True)
resampled_distribution = dict(zip(unique_resampled, counts_resampled))
print("Resampled class distribution:", resampled_distribution)

# הגדרת מסווג לוגיסטי
classifier = LogisticRegression(random_state=42, max_iter=1000, multi_class='multinomial', class_weight='balanced')

# אימון המודל
classifier.fit(X_resampled, y_resampled)

# הגדרת מדדים לקרוס ולידציה
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision_macro': make_scorer(precision_score, average='macro'),
    'recall_macro': make_scorer(recall_score, average='macro'),
    'f1_macro': make_scorer(f1_score, average='macro')
}

# ביצוע Cross Validation
cv_results = cross_validate(classifier, X_resampled, y_resampled, cv=5, scoring=scoring)

# הדפסת תוצאות קרוס ולידציה
print("\nCross-Validation Results:")
print("Accuracy:", cv_results['test_accuracy'])
print("Mean Accuracy:", cv_results['test_accuracy'].mean())
print("Precision (Macro):", cv_results['test_precision_macro'])
print("Mean Precision (Macro):", cv_results['test_precision_macro'].mean())
print("Recall (Macro):", cv_results['test_recall_macro'])
print("Mean Recall (Macro):", cv_results['test_recall_macro'].mean())
print("F1 Score (Macro):", cv_results['test_f1_macro'])
print("Mean F1 Score (Macro):", cv_results['test_f1_macro'].mean())

# חיזוי תוויות על כל קפל לצורך יצירת קונפיוז'ן מטריקס
y_pred = cross_val_predict(classifier, X_resampled, y_resampled, cv=5)

# חישוב קונפיוז'ן מטריקס
conf_matrix = confusion_matrix(y_resampled, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

# הדפסת FN, FP
fn_count = np.sum((y_resampled == 1) & (y_pred != 1))
fp_count = np.sum((y_resampled != 1) & (y_pred == 1))
print(f"False Negatives (FN): {fn_count}")
print(f"False Positives (FP): {fp_count}")

# שמירת המודל
joblib.dump(classifier, 'classifier_multiclass_model.pkl')
print("Model saved successfully!")


Unique categories in 'y_numeric': [0 1 2]
Original class distribution: {0: 766, 1: 228, 2: 69}
Sampling strategy: {0: 459, 1: 153, 2: 69}
Resampled class distribution: {0: 459, 1: 153, 2: 69}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and wi


Cross-Validation Results:
Accuracy: [0.72992701 0.75735294 0.75735294 0.75735294 0.75735294]
Mean Accuracy: 0.7518677544010305
Precision (Macro): [0.55910013 0.60419433 0.64680465 0.63254728 0.65053763]
Mean Precision (Macro): 0.6186368034870959
Recall (Macro): [0.57139518 0.63949725 0.70234645 0.67173913 0.6328725 ]
Mean Recall (Macro): 0.6435701013961884
F1 Score (Macro): [0.5640243  0.61328535 0.66052136 0.64772727 0.64075592]
Mean F1 Score (Macro): 0.6252628412382448


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and wi


Confusion Matrix:
[[381  46  32]
 [ 31 100  22]
 [ 16  22  31]]
False Negatives (FN): 53
False Positives (FP): 68
Model saved successfully!
